In this lab, we attempted to implement a text-to-speech (TTS) model based on the GPT-2 architecture. Spoiler alert — the results were not particularly successful. Therefore, we propose to walk through our approach step by step and analyze the key challenges we encountered along the way.

In [1]:
import os
import pandas as pd

from typing import List

import torch
import torch.nn as nn
from transformers import (
    GenerationMixin,
)
from transformers import GPT2Tokenizer, GPT2Model, GPT2PreTrainedModel, GPT2Config
from transformers import GenerationConfig
from transformers import EncodecModel, AutoProcessor

import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import librosa

from IPython.display import Audio, display

e:\UCU Third Year\Term 2\Audio Processing\Audio_Processing_Labs\Lab5\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
audio_folder = "./LJSpeech-1.1/wavs/"
metadata_path = "./LJSpeech-1.1/metadata.csv"

To begin, let’s briefly examine the dataset used for training our model. It consists of audio recordings paired with transcriptions — a combination that is more than sufficient for a TTS task.

In [3]:
metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['ID', 'Transcription', 'Normalized_Transcription'])
print("Metadata loaded. First few rows:")
print(metadata.head())

Metadata loaded. First few rows:
           ID                                      Transcription  \
0  LJ001-0001  Printing, in the only sense with which we are ...   
1  LJ001-0002                     in being comparatively modern.   
2  LJ001-0003  For although the Chinese took impressions from...   
3  LJ001-0004  produced the block books, which were the immed...   
4  LJ001-0005  the invention of movable metal letters in the ...   

                            Normalized_Transcription  
0  Printing, in the only sense with which we are ...  
1                     in being comparatively modern.  
2  For although the Chinese took impressions from...  
3  produced the block books, which were the immed...  
4  the invention of movable metal letters in the ...  


In [4]:
def load_audio_file(audio_id, sr=16000):
    path = os.path.join(audio_folder, f"{audio_id}.wav")
    audio, _ = librosa.load(path, sr=sr)
    return audio

def play_audio(audio_transcription_pairs, index, sr=16000):
    print(audio_transcription_pairs[index][1])
    audio = audio_transcription_pairs[index][0]
    display(Audio(audio, rate=sr))

In [5]:
audio_transcription_pairs = []
for _, row in tqdm(metadata.iterrows()):
    audio = load_audio_file(row['ID'])
    transcription = row['Transcription']
    audio_transcription_pairs.append((audio, transcription))

13100it [03:01, 72.09it/s]


In [6]:
play_audio(audio_transcription_pairs, 0)

Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition


In [7]:
play_audio(audio_transcription_pairs, 1000)

To specify more particularly one or two of the worst, it may be mentioned that in the Borough Compter


In [8]:
play_audio(audio_transcription_pairs, 5000)

He had all the characteristics of the poisoner -- the calm deliberation,


# Dataset

While the core model only requires audio and corresponding text tokens to function, we also retained the raw data for convenience.

In [9]:
class TTSDataset(Dataset):
    def __init__(self, texts, audio_files, tokenizer, processor, codec, max_length=1024):
        self.texts = texts
        self.audio_files = audio_files
        self.tokenizer = tokenizer
        self.processor = processor
        self.codec = codec
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        text_tokens = self.tokenizer(text, return_tensors='pt')['input_ids'][0]

        audio = self.audio_files[idx]
        inputs = self.processor(
            raw_audio=audio,
            sampling_rate=self.processor.sampling_rate,
            return_tensors="pt"
        )
        encoder_outputs = self.codec.encode(
            inputs["input_values"],
            inputs["padding_mask"] if "padding_mask" in inputs else None
        )
        audio_tokens = encoder_outputs.audio_codes[0, :2]

        return {
            'text_tokens': text_tokens,
            'audio_tokens': audio_tokens,
            'text': text,
            'audio': audio
        }

# Modeling

Below is the main architecture of our TTS model based on GPT-2, along with the preprocessing and decoding methods we implemented. One critical issue we encountered early on lies in the `forward` function provided with the model. This function expects input as array of strings, which cannot be moved to VRAM and so train on GPU. This design significantly complicates the training process and limits performance scalability. We will revisit this limitation later when discussing potential fixes, though it is not the only concern with this function.

In [14]:
class GPT2TTS(GPT2PreTrainedModel, GenerationMixin):
    def __init__(self, n_special_tokens=2, **kwargs):
        super().__init__(config=GPT2Config())
        self.h = None

        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # 124M
        self.model = GPT2Model.from_pretrained('gpt2')

        self.codec = EncodecModel.from_pretrained("facebook/encodec_24khz")
        self.processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

        self.loss_fn = nn.CrossEntropyLoss()

        self.n_special_tokens = n_special_tokens

        # we will use the same embedding table for all codebooks (2)
        self.codebook_size = self.codec.config.codebook_size + self.n_special_tokens
        self.audio_emb = nn.Embedding(self.codebook_size, self.model.config.hidden_size)

        self.lm_head = nn.Linear(self.model.config.hidden_size, self.codebook_size, bias=False)

        self.bos_token_id = self.codebook_size - 2
        self.eos_token_id = self.codebook_size - 1

    def can_generate(self):
        return True

    def preprocess_text_and_audio(self, text: List, audio: List) -> tuple[List, List]:

        B = len(text)
        device = audio[0].device if len(audio) > 0 and isinstance(audio[0], torch.Tensor) else 'cpu'

        text_tokens = []
        audio_tokens = []

        # tokenize text with gpt2 tokenizer, tokenize audio with encodec and flatten
        # audio tokens sequences from [2, T] to [2 * T] such that tokens from two codebooks neighboring:
        # (0, 0), (1 ,0), (0, 1), (1, 1), (0, 2), (1, 2), (0, 3), (1, 3) etc

        # ↓↓↓ YOUR CODE HERE ↓↓↓
        for i in range(B):
            encoded_text = self.tokenizer(text[i], return_tensors='pt')
            text_tokens.append(encoded_text['input_ids'][0].to(device))

            if isinstance(audio[i], torch.Tensor) and audio[i].numel() > 0:
                audio_sample = audio[i].cpu().numpy()
                if hasattr(audio[i], 'sample_rate') and audio[i].sample_rate != self.processor.sampling_rate:
                    audio_sample = librosa.resample(
                        audio_sample,
                        orig_sr=audio[i].sample_rate,
                        target_sr=self.processor.sampling_rate
                    )

                inputs = self.processor(
                    raw_audio=audio_sample,
                    sampling_rate=self.processor.sampling_rate,
                    return_tensors="pt"
                )
                encoder_outputs = self.codec.encode(
                    inputs["input_values"].to(device),
                    inputs["padding_mask"].to(device) if "padding_mask" in inputs else None
                )

                codes = encoder_outputs.audio_codes[:, :2, :].to(device)  # [B, 2, T]

                # Flatten the codes to get [2*T] as described in the task
                # We want (0,0), (1,0), (0,1), (1,1), etc.
                flattened_codes = torch.zeros(2 * codes.shape[2], dtype=torch.long, device=device)
                flattened_codes[0::2] = codes[0, 0, :]
                flattened_codes[1::2] = codes[0, 1, :]

                audio_tokens.append(flattened_codes)
            else:
                # Empty placeholder for demonstration
                audio_tokens.append(torch.tensor([], device=device, dtype=torch.long))
        # ↑↑↑ YOUR CODE HERE ↑↑↑

        return text_tokens, audio_tokens

    def forward(
        self,
        text: List = None,
        audio: List = None,
        return_loss: bool = False,
        **kwargs,
    ) -> dict[str, torch.Tensor]:
        if not return_loss:
            # inference mode
            if kwargs.get("input_ids") is not None:
                kwargs["inputs_embeds"] = self.audio_emb(kwargs["input_ids"])
                kwargs.pop("input_ids")
            outputs = self.model(**kwargs)

            hidden_states = outputs.last_hidden_state
            outputs.logits = self.lm_head(hidden_states).float()

            return outputs

        text_tokens, audio_tokens  = \
            self.preprocess_text_and_audio(text, audio)

        B = len(text_tokens)
        H = self.model.config.hidden_size
        device = text_tokens[0].device

        L = max(len(text_tokens[i]) + len(audio_tokens[i]) + 2 for i in range(B))

        bos_token = self.audio_emb.weight[self.bos_token_id, :].unsqueeze(0)
        eos_token = self.audio_emb.weight[self.eos_token_id, :].unsqueeze(0)

        x = torch.zeros(B, L, H, device=device)

        for i in range(B):
            end_text = len(text_tokens[i])
            end_audio = end_text + 1 + len(audio_tokens[i])

            x[i, 0:end_text, :] = self.model.wte(text_tokens[i])
            x[i, end_text : end_text + 1, :] = bos_token
            x[i, end_text + 1 : end_audio, :] = self.audio_emb(audio_tokens[i])
            x[i, end_audio : end_audio + 1, :] = eos_token

        outputs = self.model(
            inputs_embeds=x,
            return_dict=True,
        )
        hidden_states = outputs.last_hidden_state
        logits = self.lm_head(hidden_states).float()

        loss = None
        labels = torch.full((B, L), -100, device=device, dtype=torch.long)
        for i in range(B):
            prompt_end = len(text_tokens[i])
            audio_end = prompt_end + len(audio_tokens[i])
            labels[i, prompt_end : audio_end] = audio_tokens[i]
            labels[i, audio_end] = self.eos_token_id

        loss = self.loss_fn(
            logits.view(-1, self.codebook_size), labels.view(-1)
        )

        return {"loss": loss, "logits": logits}

    def prepare_prompt(self, text: List[str], **kwargs):
        assert len(text) == 1, "Inference supports batch size 1 only"

        # implement prompt creation from text
        # you need to tokenize text, create text embeddings and add bos token
        # from audio embeddings

        # ↓↓↓ YOUR CODE HERE ↓↓↓
        device = next(self.parameters()).device

        encoded_text = self.tokenizer(text[0], return_tensors='pt')
        text_tokens = encoded_text['input_ids'][0].to(device)

        text_embeddings = self.model.wte(text_tokens)

        bos_token_embedding = self.audio_emb.weight[self.bos_token_id, :].unsqueeze(0)

        inputs_embeds = torch.cat([text_embeddings, bos_token_embedding], dim=0)

        inputs_embeds = inputs_embeds.unsqueeze(0)
        # ↑↑↑ YOUR CODE HERE ↑↑↑

        return inputs_embeds

    def pad_audio_tokens(self, audio_tokens: torch.Tensor, n_q: int, pad_id: int = 0):
        seq_len = audio_tokens.size(-1)
        remainder = seq_len % n_q
        if remainder != 0:
            pad_len = n_q - remainder
            pad = torch.full((1, pad_len), pad_id, dtype=audio_tokens.dtype, device=audio_tokens.device)
            audio_tokens = torch.cat([audio_tokens, pad], dim=-1)
        return audio_tokens


    def decode(self, audio_tokens: torch.Tensor):
        # unflatten audio_tokens back into 2 codebooks
        # if length is odd drop last token
        # decode with codec.decode into the waveform

        # ↓↓↓ YOUR CODE HERE ↓↓↓
        if len(audio_tokens.shape) == 1:
            audio_tokens = audio_tokens.unsqueeze(0)  # add batch dimension if missing

        # Remove the last token if the length is odd
        if audio_tokens.size(-1) % 2 != 0:
            audio_tokens = audio_tokens[..., :-1]

        # Reshape from [B, 2*T] to [B, 2, T]
        codes = audio_tokens.view(audio_tokens.size(0), 2, -1)

        # Add the required dimensions for Encodec (batch, codebooks, time)
        codes = codes.unsqueeze(0)  # [1, B, 2, T]

        # Decode with Encodec
        with torch.no_grad():
            waveforms = self.codec.decode(codes, torch.ones_like(codes))

        # Return the first (and only) batch of waveforms
        waveform = waveforms[0]
        # ↑↑↑ YOUR CODE HERE ↑↑↑

        return waveform

In [15]:
tts = GPT2TTS()

Here are two examples of using the `forward` and `decode` methods with their results.


In [16]:
# train call
tts.forward(["asdf", "asdf"], [torch.tensor([]), torch.tensor([])], return_loss=True)

{'loss': tensor(7.0858, grad_fn=<NllLossBackward0>),
 'logits': tensor([[[ 1.1274e+00,  1.5037e+00,  2.3773e+00,  ...,  6.5243e-03,
           -1.4167e+00, -2.2742e-01],
          [ 2.4569e+00,  4.5778e+00,  6.6531e+00,  ..., -2.0215e-01,
           -5.1428e+00,  7.5332e-01],
          [ 4.3562e-01,  6.1627e-01,  1.0426e+00,  ..., -9.0147e-02,
           -6.4317e-01,  4.2277e-02],
          [ 6.3502e-01,  6.4148e-01,  1.0918e+00,  ..., -2.3372e-01,
           -6.3810e-01, -5.7932e-02]],
 
         [[ 1.1274e+00,  1.5037e+00,  2.3773e+00,  ...,  6.5243e-03,
           -1.4167e+00, -2.2742e-01],
          [ 2.4569e+00,  4.5778e+00,  6.6531e+00,  ..., -2.0215e-01,
           -5.1428e+00,  7.5332e-01],
          [ 4.3562e-01,  6.1627e-01,  1.0426e+00,  ..., -9.0146e-02,
           -6.4317e-01,  4.2278e-02],
          [ 6.3502e-01,  6.4148e-01,  1.0918e+00,  ..., -2.3372e-01,
           -6.3810e-01, -5.7932e-02]]], grad_fn=<UnsafeViewBackward0>)}

In [17]:
text = "The quick brown fox jumps over the lazy dog."

sampling_params = {
    "temperature": 0.4,
    "repetition_penalty": 1.25,
    "top_p": 0.8,
    "do_sample": True
}

inputs_embeds = tts.prepare_prompt([text])
audio_tokens = tts.generate(
    inputs_embeds=inputs_embeds,
    generation_config=GenerationConfig(
        **sampling_params,
        bos_token_id=tts.bos_token_id,
        eos_token_id=tts.eos_token_id,
        pad_token_id=tts.eos_token_id,
    ),
)
waveform = tts.decode(audio_tokens)

In [18]:
audio_tokens

tensor([[586, 422, 981, 659, 452, 722, 318, 722, 722]])

In [22]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
codec = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

For training, we implemented a pipeline using PyTorch Lightning. During development, we discovered an unexpected limitation: the Lightning Trainer cannot be initialized if any part of the model has already been moved to VRAM. This was especially problematic because our GPT-based model requires GPU memory initialization at construction. As a result, we frequently encountered issues when trying to launch training sessions.

In [ ]:
from torch.utils.data import DataLoader

class GPT2TTSTrainer(pl.LightningModule):
    def __init__(self, model: GPT2TTS, lr: float = 1e-4):
        super().__init__()
        self.model = model
        self.lr = lr

    def training_step(self, batch, batch_idx):
        text, audio = batch['text'], batch['audio']
        out = self.model(text=text, audio=audio, return_loss=True)
        self.log("train_loss", out["loss"])
        return out["loss"]

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=self.lr)


def train_model(epochs: int = 5, batch_size: int = 4):
    texts = []
    audio_files = []
    for audio, text in audio_transcription_pairs:
        texts.append(text)
        audio_files.append(audio)

    model = GPT2TTS()
    dataset = TTSDataset(texts, audio_files, tokenizer, processor, codec)
    
    def collate_fn(batch):
        texts = [item['text'] for item in batch]
        audios = [item['audio'] for item in batch]
        return {'text': texts, 'audio': audios}
    
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

    trainer = pl.Trainer(
        max_epochs=epochs,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1,
        log_every_n_steps=1,
        enable_progress_bar=True,
        enable_model_summary=True
    )

    trainer.fit(GPT2TTSTrainer(model), train_loader)

Unfortunately, we were unable to successfully train the model due to recurring errors, primarily rooted in the aforementioned forward function. One persistent issue involved the unintended introduction of an extra dimension during the loop:  

```python
for i in range(B):
    end_text = len(text_tokens[i])
    end_audio = end_text + 1 + len(audio_tokens[i])

    x[i, 0:end_text, :] = self.model.wte(text_tokens[i])
    x[i, end_text : end_text + 1, :] = bos_token
    x[i, end_text + 1 : end_audio, :] = self.audio_emb(audio_tokens[i])
    x[i, end_audio : end_audio + 1, :] = eos_token
```

At the time of writing, this issue remains unresolved. Despite these challenges, we have still provided functions for generating text during model evaluation, assuming pretrained weights are available or can be loaded.

In [ ]:
train_model()

We provide several examples of how to generate audio using our implementation. **Note**: since the model is not trained, the generated audio is essentially random and often unpleasant to listen to. These examples serve primarily to demonstrate the model's inference pipeline rather than its actual performance.

In [24]:
def generate_audio(model, text, max_length=500, strategy="greedy", temperature=1.0, top_k=50, top_p=0.9):
    model.eval()

    inputs_embeds = model.prepare_prompt([text])

    generation_config = {
        "max_length": max_length,
        "eos_token_id": tts.eos_token_id,
        "pad_token_id": 0,
        "do_sample": strategy != "greedy",
        "temperature": temperature,
        "top_k": top_k if strategy == "top_k" else None,
        "top_p": top_p if strategy == "nucleus" else None,
    }

    with torch.no_grad():
        outputs = model.generate(
            inputs_embeds=inputs_embeds,
            **generation_config
        )

    prompt_len = inputs_embeds.shape[1]
    audio_tokens = outputs[0, prompt_len:]

    eos_pos = (audio_tokens == model.eos_token_id).nonzero()
    if len(eos_pos) > 0:
        audio_tokens = audio_tokens[:eos_pos[0]]

    waveform = model.decode(audio_tokens)
    return waveform

texts = [
    "This is a test sentence for TTS evaluation.",
    "The quick brown fox jumps over the lazy dog.",
    "Hello world, this is a text-to-speech system."
]

strategies = ["greedy", "top_k"]
res = []
for text in texts:
    for strategy in strategies:
        audio = generate_audio(tts, text, strategy=strategy)
        audio_np = audio.detach().cpu().numpy().squeeze()
        res.append((audio_np, text, strategy))
        display(Audio(audio_np[0], rate=processor.sampling_rate))
        print(f"Generated audio for: '{text}' with {strategy} sampling")

Generated audio for: 'This is a test sentence for TTS evaluation.' with greedy sampling


Generated audio for: 'This is a test sentence for TTS evaluation.' with top_k sampling


Generated audio for: 'The quick brown fox jumps over the lazy dog.' with greedy sampling


Generated audio for: 'The quick brown fox jumps over the lazy dog.' with top_k sampling


Generated audio for: 'Hello world, this is a text-to-speech system.' with greedy sampling


Generated audio for: 'Hello world, this is a text-to-speech system.' with top_k sampling


# Evaluation

We also included functions for evaluating the model against **Whisper**, a state-of-the-art model for speech processing. However, due to the fact that our model was not successfully trained, we did not run these evaluations in practice.

In [ ]:
from transformers import (
    XttsAutoModel,
    XttsTokenizer,
    WhisperForConditionalGeneration,
    WhisperProcessor,
    pipeline
)
from pydub import AudioSegment
from jiwer import cer

xtts_model = XttsAutoModel.from_pretrained("coqui/XTTS-v2")
xtts_tokenizer = XttsTokenizer.from_pretrained("coqui/XTTS-v2")

whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

utmos = pipeline("audio-quality-assessment", model="tarepan/utmos-strong")

speaker_verification = pipeline("speaker-verification", model="speechbrain/spkrec-ecapa-voxceleb")

In [ ]:
def compute_cer(reference_text, generated_audio_path):
    audio, sr = librosa.load(generated_audio_path, sr=16000)
    inputs = whisper_processor(audio, sampling_rate=sr, return_tensors="pt")
    predicted_ids = whisper_model.generate(**inputs)
    predicted_text = whisper_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    cer_score = cer(reference_text, predicted_text)
    return cer_score


def compute_utmos(generated_audio_path):
    audio, sr = librosa.load(generated_audio_path, sr=24000)
    audio = torch.from_numpy(audio).unsqueeze(0)

    utmos_score = utmos(audio.numpy(), sampling_rate=sr)[0]["score"]
    return utmos_score


def compute_secs(reference_audio_path, generated_audio_path):
    ref_audio, _ = librosa.load(reference_audio_path, sr=16000)
    gen_audio, _ = librosa.load(generated_audio_path, sr=16000)

    ref_embedding = speaker_verification(ref_audio)["embedding"]
    gen_embedding = speaker_verification(gen_audio)["embedding"]

    secs = torch.nn.functional.cosine_similarity(
        torch.tensor(ref_embedding),
        torch.tensor(gen_embedding),
        dim=0
    ).item()

    return secs


def evaluate_tts(
    reference_text: str,
    reference_audio_path: str,
    generated_audio_path: str,
) -> dict:
    """Runs full evaluation (CER, UTMOS, SECS)"""
    cer_score = compute_cer(reference_text, generated_audio_path)
    utmos_score = compute_utmos(generated_audio_path)
    secs_score = compute_secs(reference_audio_path, generated_audio_path)

    return {
        "CER": cer_score,
        "UTMOS": utmos_score,
        "SECS": secs_score,
    }

In [ ]:
# Example inputs
reference_text = "The quick brown fox jumps over the lazy dog."
reference_audio_path = "ref_audio.wav"  # Ground truth audio
generated_audio_path = "gen_audio.wav"  # Your GPT-2 TTS output

# Run evaluation
results = evaluate_tts(reference_text, reference_audio_path, generated_audio_path)

print(f"""
Evaluation Results:
- CER: {results['CER']:.4f}  (Lower is better)
- UTMOS: {results['UTMOS']:.2f}/5  (Higher is better)
- SECS: {results['SECS']:.4f}/1  (Higher is better)
""")

# Forward update

In this section, we made a partial fix to the `forward` method. It now accepts preprocessed token inputs instead of raw data arrays, making it usable for GPU-based training. However, one significant issue still persists: the introduction of an extra dimension in the loop, which breaks training due to shape mismatches. This remains unresolved and was a major blocker for successful model training.

In [ ]:
class GPT2TTS(GPT2PreTrainedModel, GenerationMixin):
    def __init__(self, n_special_tokens=2, **kwargs):
        ...

    def forward(self, text_tokens=None, audio_tokens=None, return_loss=False, **kwargs):
        if not return_loss:
            # Inference mode
            if kwargs.get("input_ids") is not None:
                kwargs["inputs_embeds"] = self.audio_emb(kwargs.pop("input_ids"))
            outputs = self.model(**kwargs)
            hidden_states = outputs.last_hidden_state
            logits = self.lm_head(hidden_states).float()
            outputs.logits = logits
            return outputs

        # Training mode with given tokens
        B = len(text_tokens)
        H = self.model.config.hidden_size
        device = text_tokens[0].device

        L = max(len(text_tokens[i]) + len(audio_tokens[i]) + 3 for i in range(B))

        bos_token = self.audio_emb.weight[self.bos_token_id].unsqueeze(0)
        eos_token = self.audio_emb.weight[self.eos_token_id].unsqueeze(0)

        x = torch.zeros(B, L, H, device=device)

        # TODO: Fix bug with dimensanality here
        for i in range(B):
            end_text = len(text_tokens[i])
            end_audio = end_text + 1 + len(audio_tokens[i])

            x[i, 0:end_text, :] = self.model.wte(text_tokens[i])
            x[i, end_text : end_text + 1, :] = bos_token
            x[i, end_text + 1 : end_audio, :] = self.audio_emb(audio_tokens[i])
            x[i, end_audio : end_audio + 1, :] = eos_token

        outputs = self.model(inputs_embeds=x, return_dict=True)
        hidden_states = outputs.last_hidden_state
        logits = self.lm_head(hidden_states).float()

        labels = torch.full((B, L), -100, device=device, dtype=torch.long)
        for i in range(B):
            prompt_end = len(text_tokens[i])
            audio_end = prompt_end + len(audio_tokens[i])
            labels[i, prompt_end:audio_end] = audio_tokens[i]
            labels[i, audio_end] = self.eos_token_id

        loss = self.loss_fn(logits.view(-1, self.codebook_size), labels.view(-1))

        return {"loss": loss, "logits": logits}

# Conclusion

Overall, this lab work was very interesting and at the same time painful and offered an opportunity to work with modern generative models in a speech synthesis context. However, it became clear during implementation that the provided model and codebase feel untested, particularly due to design issues with key functions. 

For future iterations of this lab, we strongly recommend:
- Including practical tips or documentation on model usage, architecture, and training strategies (e.g., which LLM variant to use, typical training time, VRAM requirements)
- Allocating more time for this lab, as training generative models—even small ones—is computationally expensive and time-consuming.

Despite the setbacks, we genuinely appreciate the opportunity to work on this lab and study this course and would like to thank the course team for such a challenging and interesting course!